In [1]:
from spark import *
execute_spark()

25/04/19 22:23:31 WARN Utils: Your hostname, MacBook-Air-de-Igoh.local resolves to a loopback address: 127.0.0.1; using 172.17.1.180 instead (on interface en0)
25/04/19 22:23:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/igoh/.ivy2/cache
The jars for the packages stored in: /Users/igoh/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7f55e967-a9ac-4cbf-ad72-2f1dcdc4f147;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 109ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

CodeCache: size=131072Kb used=32289Kb max_used=32406Kb free=98782Kb
 bounds [0x000000010b1f8000, 0x000000010d1c8000, 0x00000001131f8000]
 total_blobs=13715 nmethods=12715 adapters=910
 compilation: disabled (not enough contiguous free space left)


In [2]:
spark.sql("""select * from bd_becomex.import where 	CO_ANO = '2025'""").toPandas()

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO
0,2025,1,29342020,10,160,SP,1,817800,36040,36040,94348,5999,37
1,2025,2,29214410,10,361,SP,4,817700,2,2,385,121,0
2,2025,3,90221419,11,361,MG,4,817700,1,541,61192,5051,17
3,2025,1,85423120,11,161,AM,1,227600,665278,624,2680710,661,883
4,2025,3,85258929,11,386,SP,4,817600,1,0,1647,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
585814,2025,2,91139000,10,160,RJ,1,717600,12,12,900,11,0
585815,2025,3,82083000,10,249,MS,4,817600,10,10,883,546,0
585816,2025,1,73083000,10,160,MG,1,817800,407,407,16492,220,0
585817,2025,2,28112230,10,232,RN,4,317901,400,400,988,1448,1


In [4]:
spark.sql("""use bd_becomex""")

DataFrame[]

In [5]:
df = spark.sql("""

WITH ncm_joined AS (
    SELECT *
    FROM ncm n
    INNER JOIN sh s ON n.CO_SH6 = s.CO_SH6
    INNER JOIN ncm_ppe np ON n.CO_PPE = np.CO_PPE
    INNER JOIN ncm_ppi npi ON n.CO_PPI = npi.CO_PPI
    INNER JOIN ncm_fat_agreg nfa ON n.CO_FAT_AGREG = nfa.CO_FAT_AGREG
    INNER JOIN ncm_cuci nc ON n.CO_CUCI_ITEM = nc.CO_CUCI_ITEM
    INNER JOIN ncm_cgce ncg ON n.CO_CGCE_N3 = ncg.CO_CGCE_N3
    INNER JOIN ncm_isic ni ON n.CO_ISIC_CLASSE = ni.CO_ISIC_CLASSE
),
fat_import_joined AS (
    SELECT *
    FROM bd_becomex.import fi
    INNER JOIN ncm_unidade nu ON fi.CO_UNID = nu.CO_UNID
    INNER JOIN urf u ON fi.CO_URF = u.CO_URF
    INNER JOIN paises p ON fi.CO_PAIS = p.CO_PAIS
    INNER JOIN via v ON fi.CO_VIA = v.CO_VIA
    WHERE fi.CO_ANO = 2025
)

SELECT *
FROM fat_import_joined fi
INNER JOIN ncm_joined nj ON fi.CO_NCM = nj.CO_NCM
WHERE fi.CO_ANO = '2025'

""")#.toPandas()


In [ ]:
df = spark.sql("""

WITH ncm_joined AS (
    SELECT *
    FROM ncm n
    INNER JOIN sh s ON n.CO_SH6 = s.CO_SH6
    INNER JOIN ncm_ppe np ON n.CO_PPE = np.CO_PPE
    INNER JOIN ncm_ppi npi ON n.CO_PPI = npi.CO_PPI
    INNER JOIN ncm_fat_agreg nfa ON n.CO_FAT_AGREG = nfa.CO_FAT_AGREG
    INNER JOIN ncm_cuci nc ON n.CO_CUCI_ITEM = nc.CO_CUCI_ITEM
    INNER JOIN ncm_cgce ncg ON n.CO_CGCE_N3 = ncg.CO_CGCE_N3
    INNER JOIN ncm_isic ni ON n.CO_ISIC_CLASSE = ni.CO_ISIC_CLASSE
),
fat_import_joined AS (
    SELECT *
    FROM bd_becomex.import fi
    INNER JOIN ncm_unidade nu ON fi.CO_UNID = nu.CO_UNID
    INNER JOIN urf u ON fi.CO_URF = u.CO_URF
    INNER JOIN paises p ON fi.CO_PAIS = p.CO_PAIS
    INNER JOIN via v ON fi.CO_VIA = v.CO_VIA
),
fat_export_joined AS (
    SELECT *
    FROM bd_becomex.export fe
    INNER JOIN ncm_unidade nu ON fe.CO_UNID = nu.CO_UNID
    INNER JOIN urf u ON fe.CO_URF = u.CO_URF
    INNER JOIN paises p ON fe.CO_PAIS = p.CO_PAIS
    INNER JOIN via v ON fe.CO_VIA = v.CO_VIA
)
SELECT *
FROM fat_import_joined fi
INNER JOIN ncm_joined nj ON fi.CO_NCM = nj.CO_NCM
WHERE fi.CO_ANO = '2024'

""").toPandas()


In [10]:
df = df.drop("CO_CGCE_N3")
df = df.drop("co_cuci_item")
df = df.drop("co_fat_agreg")
df = df.drop("co_isic_classe")
df = df.drop("CO_PPI")
df = df.drop("CO_SH6")
df = df.drop("co_ncm")
df = df.drop("CO_UNID")
df = df.drop("CO_URF")
df = df.drop("CO_PAIS")
df = df.drop("co_ppe")


df.write.mode("overwrite").parquet("arquivo.parquet")
